In [8]:
from langchain_groq import ChatGroq
from langchain_chroma import Chroma
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
import os
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
groq_api_key = os.getenv('GROQ_API_KEY')
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN')

In [5]:
llm = ChatGroq(model='llama3-8b-8192',groq_api_key=groq_api_key)
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7fd2a1ef3460>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7fd2a1ef22f0>, model_name='llama3-8b-8192')

In [6]:
embedding = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')

/official/official_docs/PythonLearning/DeepLearning/nlp/Projects/chatbot-deployment/venv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/official/official_docs/PythonLearning/DeepLearning/nlp/Projects/chatbot-deployment/venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

In [10]:
vector_store = Chroma.from_documents(documents,embedding=embedding)
vector_store

In [11]:
vector_store.similarity_search("cat",k=1)

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')]

In [12]:
# async query
await vector_store.asimilarity_search("dog",k=2)

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]

In [15]:
# Using Retriver with vector_store. We can not directly use vector_store directly in LECL
from typing import List
from langchain_core.runnables import RunnableLambda

retriever = RunnableLambda(vector_store.similarity_search).bind(k=1)
retriever.batch(['cat','dog'])

[[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]]

In [17]:
# This is the best way to query from a vector store
retriver2 = vector_store.as_retriever(search_type='similarity',search_kwargs={'k':1})
retriver2.batch(['cat','dog'])

[[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]]

In [18]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = '''Answer the question using provided context only.
{question}
Contxt:
{context}
'''
prompt = ChatPromptTemplate.from_messages([('human',message)])
prompt

ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='Answer the question using provided context only.\n{question}\nContxt:\n{context}\n'))])

In [19]:
rag_chain = {'context':retriver2,'question':RunnablePassthrough()}| prompt | llm 
rag_chain

{
  context: VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x7fd1ce6f9f60>, search_kwargs={'k': 1}),
  question: RunnablePassthrough()
}
| ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='Answer the question using provided context only.\n{question}\nContxt:\n{context}\n'))])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7fd2a1ef3460>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7fd2a1ef22f0>, model_name='llama3-8b-8192')

In [20]:
rag_chain.invoke("Tell me about Parrots")

AIMessage(content='According to the provided context, Parrots are intelligent birds capable of mimicking human speech.', response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 55, 'total_tokens': 74, 'completion_time': 0.015833333, 'prompt_time': 0.006654431, 'queue_time': 0.006893348, 'total_time': 0.022487764}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_a97cfe35ae', 'finish_reason': 'stop', 'logprobs': None}, id='run-c935027f-b735-4f31-8361-fb649b7ca30f-0', usage_metadata={'input_tokens': 55, 'output_tokens': 19, 'total_tokens': 74})